In [22]:
import warnings
warnings.filterwarnings('ignore')


In [23]:

from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer

import pandas as pd
%matplotlib inline

from collections import OrderedDict
from pathlib import Path

import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import pyLDAvis
from pyLDAvis.sklearn import prepare

from wordcloud import WordCloud
from termcolor import colored

# sklearn for feature extraction & modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import joblib

import re

%matplotlib inline
import sys
from pathlib import Path

import pandas as pd

import spacy
from spacy import displacy
from textacy.extract import ngrams, entities
nlp = spacy.load('en_core_web_lg') 


In [24]:
# validate installation
!{sys.executable} -m spacy validate


✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.4.1) =================
ℹ spaCy installation:
c:\Users\JBarr\anaconda3\envs\blockchainDev\lib\site-packages\spacy

NAME              SPACY            VERSION    
en_core_web_lg    >=3.4.0,<3.5.0   3.4.0     ✔
en_core_web_md    >=3.4.0,<3.5.0   3.4.0     ✔
en_core_web_sm    >=3.4.0,<3.5.0   3.4.0     ✔
es_core_news_sm   >=3.4.0,<3.5.0   3.4.0     ✔



In [7]:

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

stop = stopwords.words('english')


In [8]:

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text


def get_attributes(f):
    print([a for a in dir(f) if not a.startswith('_')], end=' ')

<>:3: DeprecationWarning: invalid escape sequence \)
<>:5: DeprecationWarning: invalid escape sequence \W
<>:3: DeprecationWarning: invalid escape sequence \)
<>:5: DeprecationWarning: invalid escape sequence \W
C:\Users\JBarr\AppData\Local\Temp\ipykernel_167872\1556330046.py:3: DeprecationWarning: invalid escape sequence \)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
C:\Users\JBarr\AppData\Local\Temp\ipykernel_167872\1556330046.py:5: DeprecationWarning: invalid escape sequence \W
  text = (re.sub('[\W]+', ' ', text.lower()) +


In [9]:

datadf = pd.DataFrame(pd.read_csv('movie_data.csv'),
                      columns=['review', 'sentiment'])


In [10]:
datadf['review'] = datadf['review'].apply(preprocessor)

X_train = datadf.loc[:25000, 'review'].values
y_train = datadf.loc[:25000, 'sentiment'].values
X_test = datadf.loc[25000:, 'review'].values
y_test = datadf.loc[25000:, 'sentiment'].values

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

In [12]:
#gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x000001DAFF415DC0>,
                                              <function tokenizer_porter at 0x000001DABA7C8040>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [13]:

print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001DAFF415DC0>} 
CV Accuracy: 0.897


In [14]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))


Test Accuracy: 0.899


In [15]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:

datadf = pd.DataFrame(pd.read_csv('movie_data.csv'),
                      columns=['review', 'sentiment'])


In [ ]:

count = CountVectorizer(stop_words='english',
                        max_features=5000)
X = count.fit_transform(datadf['review'].values)


In [18]:
#%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))


NameError: name '__file__' is not defined

In [16]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

<>:3: DeprecationWarning: invalid escape sequence \)
<>:5: DeprecationWarning: invalid escape sequence \W
<>:3: DeprecationWarning: invalid escape sequence \)
<>:5: DeprecationWarning: invalid escape sequence \W
C:\Users\JBarr\AppData\Local\Temp\ipykernel_167872\3487550917.py:3: DeprecationWarning: invalid escape sequence \)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
C:\Users\JBarr\AppData\Local\Temp\ipykernel_167872\3487550917.py:5: DeprecationWarning: invalid escape sequence \W
  text = re.sub('[\W]+', ' ', text.lower()) \


In [19]:
import os


vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

os.chdir('movieclassifier')

In [20]:
# change to your data path if necessary
DATA_DIR = Path('./data/')
data_path = DATA_DIR / 'earnings_calls'
results_path = Path('results')
model_path = Path('results', 'earnings_calls')
if not model_path.exists():
    model_path.mkdir(exist_ok=True, parents=True)
    

In [21]:
files = pd.read_csv(list(data_path.glob('**/*.csv')))
doc_list = []
for i  in (files):
    with open(str(i), encoding='latin1') as f:
        topic = i.parts[-2]
        lines = f.readlines()
        heading = lines[0].strip()
        body = ' '.join([l.strip() for l in lines[1:]])
        doc_list.append([topic.capitalize(), heading, body])

ValueError: Invalid file path or buffer object type: <class 'list'>

In [ ]:
Dfdocs = pd.DataFrame(doc_list, columns=['Ticker', 'heading', 'article'])


In [ ]:
Dfdocs

,Ticker,heading,article


In [22]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ("I love this movie. It's amazing.",
           'Happy life')
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))

NameError: name 'vect' is not defined